# Tarea 3: Regularización y Generalización <br/> CC6204 Deep Learning, Universidad de Chile 

**Fecha de entrega: 1 de octubre de 2019 ([Hoja de respuestas](https://colab.research.google.com/drive/1S-uK3siwBUpPnka_IGGXqFT3lM6v8c2T))**

En esta tarea programarás distintos métodos de regularización para las redes que construiste en la [Tarea 2](https://colab.research.google.com/drive/1ZnrSZQDhDrmp6mwIxqb_NWlXB77Til7c) (si tuviste problemas resolviendo la Tarea 2, puedes usar [esta solución](https://colab.research.google.com/drive/16Nth5zgJKWrqywrO8yErd9xrLKd2Hb6c) gentileza de Jesús Pérez). Además entrenarás tu red en MNIST usando regularización y estimarás su desempeño usando un conjunto de prueba.

Te recomendamos que repases la materia de las clases de regularización. El material necesario para resolver esta tarea es el siguiente:
* [Video: Redes Neuronales en la Práctica](https://youtu.be/ML5jnWMzCoA?t=2802) (desde el minuto 47)
* [Video: Regularización](https://www.youtube.com/watch?v=MmVY_ZLWF0E) 

IMPORTANTE: A menos que se exprese lo contrario, sólo podrás utilizar las clases y funciones en el módulo [`torch`](https://pytorch.org/docs/stable/torch.html).

(por Jorge Pérez, https://github.com/jorgeperezrojas, [@perez](https://twitter.com/perez))

# Parte 1: Regularización por norma $L_2$

## 1a) Penalización de norma $L_2$ en la pérdida

Extiende tu implementación de la clase `FFNN` para recibir el parámetro opcional `l2_par` al momento de inicializar la clase. Cuando este parámetro sea distinto de `None` tu red debe entrenarse usando la función de pérdida
\begin{equation}
  \mathcal{L} = \textit{CELoss} + \frac{\alpha}{2N}\sum_{i=1}^{L}\|W^{(i)}\|_F^2+\frac{\alpha}{2N}\|U\|_F^2
\end{equation}
donde $N$ es el tamaño del minibatch usado en el entrenamiento, $\alpha$ es el paramétro `l2_par`, $W^{(1)},\ldots,W^{(L)},U$ son los parámetros de nuestra red (sin considerar los parámetros de sesgo) y $\|A \|_F^2$ denota la suma de los cuadrados de todos los valores de una matriz $A$.

Nota que no debes actualizar la pasada hacia adelante de tu red (la función `forward`) pero sí debes actualizar la forma de calcular el gradiente en cada caso (función `backward`) y también la función de error (de hecho, tendrás que "avisarle" de alguna forma a tu función que computa la pérdida que estás usando regularización $L_2$).

## 1b) Regularización por *weight decay*

Una manera alternativa (y posiblemente más fácil de programar) la regularización anterior es con el concepto de *weight decay*. Para esto simplemente debes modificar tu clase de descenso estocástico de gradiente `SGD` de manera que el inicializador reciba un nuevo argumento `weight_decay`. La idea es que en cada paso de actualización de los parámetros, antes de actualizarlos los multipliques por el `1 - weight_decay`. 

# Parte 2: Regularización por dropout

Implementa el método de dropout para apagar aleatoriamente ciertas neuronas de tu red. Para esto el inicializador de tu red debe recibir otro parámetro opcional que llamaremos `keep_prob` que será una lista de probabilidades que indicarán la probabilidad de mantener las neuronas de cada capa. La lista debe tener largo $L+1$ e indicará las probabilidades incluyendo la capa de input. Por ejemplo, si tu red es inicializada como

```
red_neuronal = FFNN(300,[50,30],[relu,sig],10, l2_par=None, keep_prob=[1.0, 0.5, 0.7])
```

esto quiere decir que en la implementación de dropout, las neuronas de la capa de input deben mantenerse con probabilidad $1.0$ (o sea, nunca deben borrarse), las neuronas de la primera capa escondida deben mantenerse con probabilidad $0.5$ y las de la segunda capa escondida deben mantenerse con probabilidad $0.7$.

Debes modificar los métodos `forward` y `backward` para que ambos consideren las probabilidades de dropout. Agrega también un parámetro booleano opcional `predict` a la función `forward` que asegure que las probabilidades de dropout no se consideren en una pasada hacia adelante que se haga para testear la red (y no para entrenarla).

Recuerda que la mejor forma de implementar dropout es con la técnica de "inverted dropout" que lo que hace es elegir para la capa $i$ una máscara de bits $M^{(i)}$ de acuerdo a la probabilidad $p_i$ de mantener encendidas las neuronas de esa capa, y luego actualiza la capa escondida como 
\begin{equation*}
h^{(i)}:=h^{(i)} * \frac{M^{(i)}}{p_i}.
\end{equation*}
Dada esta implementación, cuando se haga el forward con la opción `predict` simplemente se puede considerar que cada $p_i$ es $1$ (o sea, todas las neuronas están encendidas).

# Parte 3: Entrenamiento y generalización sobre MNIST 

Usa tu red neuronal para entrenar con los datos de MNIST. Debes entrenar usando el conjunto de entrenamiento (train set) y probar usando el conjunto de prueba (test set). 

En este caso nos importa sobre todo el aplicar métodos de regularización a la red para mejorar la generalización (disminuir la diferencia entre error en entrenamiento y error en prueba). Entrena y grafica cómo varía la pérdida y el acierto en el conjunto de entrenamiento y en el conjunto de prueba primero sin usar regularización, y luego añade regularización para obtener una mejor generalización. Reporta al menos dos configuraciones que usen regularización y discute acerca de cómo varía la pérdida y el acierto en el conjunto de prueba comparado con cuando no usas regularización. 

# Parte 4 (Opcional): Aumento de datos sintéticamente

Piensa en formas en que puedes hacer *data augmentation* con los datos de entrenamiento de MNIST e implementa todas las que quieras para mejorar aun más la generalización de tu modelo. Reporta cómo cambia el error de prueba y compáralo con no usar *data augmentation*.